In [ ]:
# 

print("STARTING NAKO.")
print('Reading Volumes.....')
file_to_read1 = './datasets/nako/100017/100017_3D_GRE_TRA_opp/3D_GRE_TRA_opp_3D_GRE_TRA_1.nii.gz'
file_to_read2 = './datasets/nako/100017/100017_3D_GRE_TRA_opp/3D_GRE_TRA_opp_3D_GRE_TRA_2.nii.gz'
file_to_read3 = './datasets/nako/100017/100017_3D_GRE_TRA_opp/3D_GRE_TRA_opp_3D_GRE_TRA_3.nii.gz'
file_to_read4 = './datasets/nako/100017/100017_3D_GRE_TRA_opp/3D_GRE_TRA_opp_3D_GRE_TRA_4.nii.gz'

img1_ras = read_ras(file_to_read1)
img2_ras = read_ras(file_to_read2)
img3_ras = read_ras(file_to_read3)
img4_ras = read_ras(file_to_read4)

print('Merging/Stitching Images.....')
img_ras_sampled_combined = multi_vol_stitching([img1_ras, img2_ras, img3_ras, img4_ras])

print('Viewing Stitched Images.....')
volume_3_view_viewer(get_volume_data(img_ras_sampled_combined))

print('Saving Processed & Stitched Image.....')
save_volume(img_ras_sampled_combined, 'nako_ras_sampled_stitched')
print('FINISHED.')

In [ ]:
print("STARTING NAKO LABEL-MAPS.")
print('Reading Label Maps.....')
label_files_to_read_list = glob.glob('./datasets/nako/100017_label/**')

img_ras_list = []
volume = img_ras_sampled_combined
for label_file_to_read in label_files_to_read_list:
    img_ras, lidx, labelname = read_ras(label_file_to_read, is_label=True)
    print(img_ras.shape, label_file_to_read, lidx, labelname)
    mode='nearest'
    img_ras = resample_to_output(img_ras, TARGET_RESOLUTION, order=3, mode=mode, cval=0.0)
#     img_ras = resample_from_to(img_ras, [volume.shape, img_ras.affine])
    print(img_ras.shape)
    img_ras_list.append([img_ras, lidx, labelname])


In [ ]:
volume = img_ras_sampled_combined
volume, stitched_label = label_stitching(img_ras_list, volume)

print('Viewing Stitched Images.....')
volume_3_view_viewer(get_volume_data(volume))
volume_3_view_viewer(get_volume_data(stitched_label))

print('Saving Processed & Stitched Image.....')
save_volume(volume, 'nako_volume')
save_volume(stitched_label, 'nako_label')
print('FINISHED.')



In [ ]:
def do_nibabel_transform_to_ras(img):
    print(f'Transforming Images to {DEFAULT_ORIENTATION}.....')
    affine = img.affine
    orig_ornt = nb.io_orientation(affine)
    targ_ornt = axcodes2ornt(DEFAULT_ORIENTATION)
    transform = ornt_transform(orig_ornt, targ_ornt)
    img = img.as_reoriented(transform)
    return img

def multi_vol_stitching(images, is_label=False):
    if len(images)==1:
        return images
    elif len(images) ==0:
        raise Exception("Empty Image List!")
        
    images_sorted = sorted(images, key=lambda im: im.header['qoffset_z'], reverse=True)
    img_0 = images_sorted[0]
    
    mode = 'nearest' if is_label else 'constant'
    img_0 = resample_to_output(img_0, TARGET_RESOLUTION, order=3, mode=mode, cval=0.0)
    
    for idx, img_1 in enumerate(images_sorted[1:]):
        print(f'{idx}th img for stitching...')
#         print("STARTTTTT: ", img_1.affine)
        img_1 = resample_to_output(img_1, TARGET_RESOLUTION, order=3, mode=mode, cval=0.0)
#         img_1 = placing_axes(img_1, img_0.affine.copy(), img_0.header.copy(), [2])
#         print( img_1.affine)
        target_affine = img_0.affine.copy()
        target_affine[2,3] = img_1.affine[2,3].copy()
        target_shape = img_0.shape[:2] + img_1.shape[2:]
#         img_1 = placing_axes(img_1, target_affine, img_0.header.copy(), skip_axis=[2])
        img_1 = resample_from_to(img_1, [target_shape, target_affine])
#         print(img_1.affine)
        img_0 = vol_stitching(img_0, img_1)
#         print(img_0.affine, 'FINISHHHHHHHHH')
        
    return img_0
    
def vol_stitching(im_0, im_1):
    im_0_z = im_0.shape[2]
    im_1_z = im_1.shape[2]

    # calculate overlap region:
    im_0_end = im_0.header['qoffset_z']
    im_1_end = im_1.header['qoffset_z']

    spacing = im_0.header['pixdim'][3]

    im_0_width = im_0_z * spacing
    im_1_width = im_1_z * spacing

    im_1_start = im_1_end + im_1_width
    im_0_start = im_0_end + im_0_width

    overlap = abs(im_0_end - im_1_start)

    overlap_v = int(round(overlap / spacing))

    new_im_dim = abs(round((abs(im_1_end - im_0_start)) / spacing))

    new_img = np.empty([im_0.shape[0], im_0.shape[1], int(new_im_dim)])

    im_0_data = im_0.get_fdata()
    im_1_data = im_1.get_fdata()

    new_img[:, :, 0:(im_1_z - overlap_v)] = im_1_data[:, :, 0:(im_1_z - overlap_v)]
    new_img[:, :, im_1_z:] = im_0_data[:, :, overlap_v:]
    
    # overlap region:
    sigmoid_c = sigmoid(np.linspace(-DEFAULT_LINSPACE, DEFAULT_LINSPACE, overlap_v))
    
    for l in range(0, overlap_v):
        new_img[:, :, (im_1_z - overlap_v + l)] = \
        (1 - sigmoid_c[l]) * im_1_data[:,:, (im_1_z - overlap_v) + l] + (sigmoid_c[l]) * im_0_data[:, :, l]
        
    stitched_img = nb.Nifti1Image(new_img, im_1.affine, im_1.header)
#     placing_axes(stitched_img)
    return stitched_img

def read_ras(file_path, file_type=None, is_label=False):
    _, _, img = file_reader(file_path, file_type)
    img_ras = do_nibabel_transform_to_ras(img)
    if is_label:
        file_labels = list(FILE_TO_LABEL_MAP.keys())
        lidx, labelname = fetch_class_labels_from_filemap(file_path, file_labels)
        return img_ras, lidx, labelname
    else:
        return img_ras

def fetch_class_labels_from_filemap(labelmap_path, file_labels):
    label_idx, label = None, None
    for lidx, file_label in enumerate(file_labels):
        if file_label in labelmap_path.replace(" ", "").upper():
            label_idx, label = lidx, file_label
            break
    return label_idx, label


def placing_axes(vol, target_affine, target_header=None, skip_axis=None):
    
    vol2target = npl.inv(target_affine).dot(vol.affine)
    source_data = vol.get_fdata()
    shifts = tuple(vol2target[:3, 3].astype(np.int32))
    print(shifts)
#     print(source_data.shape)
    for ax, shift in enumerate(shifts):
        if skip_axis is not None and ax in skip_axis:
            continue
        print(ax, shift)
        shift = int(shift)
        if shift < 0:
            source_data = flip_axis(source_data, axis=ax)
        print(-np.abs(shift))
        source_data = np.roll(source_data, -np.abs(shift), axis=ax)
                
    if target_header is None:
        target_header = nb.Nifti1Header()
    stitched_labeled_img = nb.Nifti1Image(source_data, target_affine, target_header)

    return stitched_labeled_img


def label_stitching(label_parts, img_reference):
    header = img_reference.header
    target_affine = img_reference.affine
    steps = header['pixdim'][1:4]
    volume = img_reference.get_fdata()
    im_dim_v = volume.shape
    
    processed_segm = None#np.zeros((500,500,1000))

    im_offx = abs(header['qoffset_x'])
    im_offy = abs(header['qoffset_y'])
    im_offz = abs(header['qoffset_z'])
    q_offsets = [im_offx,im_offy,im_offz]
    
    reference_labelmap = None
    for labelmap_img, lidx, lname in label_parts:
        print(lidx, lname)

        if reference_labelmap is None:
            reference_labelmap = labelmap_img
        else:
            labelmap_img = resample_from_to(labelmap_img, [reference_labelmap.shape, reference_labelmap.affine])
            print('c shape:', labelmap_img.shape, labelmap_img.affine)
            
        label_header = labelmap_img.header
        labelmap = labelmap_img.get_fdata()
        labelmap_affine = labelmap_img.affine
        steps_l = label_header['pixdim'][1:4]
        lblmp_offx = abs(label_header['qoffset_x'])
        lblmp_offy = abs(label_header['qoffset_y'])
        lblmp_offz = abs(label_header['qoffset_z'])
        q_offsets_l = [lblmp_offx,lblmp_offy,lblmp_offz]
        
        print(f'vol offsets: {q_offsets}')
        
        print(f'label_offsets: {q_offsets_l}')
        
#         l_vol = np.prod(labelmap.shape)
        segm_dim_v = labelmap.shape
        
        ############
        labelmap2vol = npl.inv(target_affine).dot(labelmap_affine)
        seg_start_inv = np.floor(apply_affine(labelmap2vol, [0,0,0])).astype(np.int32)
        seg_end_inv = apply_affine(labelmap2vol, segm_dim_v).astype(np.int32)
        print("seg start inv v: ",seg_start_inv , "segm end inv v:",seg_end_inv)
        ############
#         shifts = tuple(np.floor(labelmap2vol[:3, 3]).astype(np.int32))
        for ax, shift in enumerate(seg_start_inv):
            print(ax, shift)
            shift = int(shift)
            if ax!=2:
#                 labelmap = flip_axis(labelmap, axis=ax)
                if shift < 0:
                    labelmap = flip_axis(labelmap, axis=ax)
                    seg_end_inv[ax] += abs(seg_start_inv[ax])
                    seg_start_inv[ax] = 0
                else:
#                     labelmap = flip_axis(labelmap, axis=ax)
                    seg_end_inv[ax] -= abs(seg_start_inv[ax])
                    seg_start_inv[ax] = 0
            
        print("updated seg start inv v: ",seg_start_inv , "updated segm end inv v:",seg_end_inv)
        labelmap = np.multiply(lidx, labelmap)
        
        if processed_segm is None:
            seg_vol = np.prod(seg_end_inv)
            vol_vol = np.prod(volume.shape)
            processed_segm = np.empty(seg_end_inv) if seg_vol>vol_vol else np.zeros(volume.shape)
            
        processed_segm[seg_start_inv[0]:seg_end_inv[0], seg_start_inv[1]:seg_end_inv[1], seg_start_inv[2]:seg_end_inv[2]] += labelmap
        print("###############################################################################################")

#     print(volume.shape, processed_segm.shape)
#     processed_segm = np.flip(processed_segm)
    labelmap = np.round(processed_segm)
    empty_header = nb.Nifti1Header()
#     processed_segm_img = nb.Nifti1Image(processed_segm, target_affine, empty_header)
#     processed_segm = resample_from_to(processed_segm_img, [volume.shape, target_affine])
    
    
#     volume_ = np.zeros_like(processed_segm)
#     vx, vy, vz = volume.shape
#     volume_[0:vx, 0:vy, 0:vz] = volume
#     volume = volume_
#     volume = np.flip(np.moveaxis(volume, 2, 1))  # 012 -> 021
#     labelmap = np.flip(np.moveaxis(labelmap, 2, 1))
    volume = normalise_data(volume)

    
    stitched_labeled_img = nb.Nifti1Image(labelmap, target_affine, empty_header)
    volume_img = nb.Nifti1Image(volume, target_affine, empty_header)
    
    return volume_img, stitched_labeled_img
    

In [ ]:
# WORKS FOR KORA COPY... SLIGHTLY DIFFERENT FROM ABOVE

In [ ]:
def do_nibabel_transform_to_ras(img):
    print(f'Transforming Images to {DEFAULT_ORIENTATION}.....')
    affine = img.affine
    orig_ornt = nb.io_orientation(affine)
    targ_ornt = axcodes2ornt(DEFAULT_ORIENTATION)
    transform = ornt_transform(orig_ornt, targ_ornt)
    img = img.as_reoriented(transform)
    return img

def multi_vol_stitching(images, is_label=False):
    if len(images)==1:
        return images
    elif len(images) ==0:
        raise Exception("Empty Image List!")
        
    images_sorted = sorted(images, key=lambda im: im.header['qoffset_z'], reverse=True)
    img_0 = images_sorted[0]
    
    mode = 'nearest' if is_label else 'constant'
    img_0 = resample_to_output(img_0, TARGET_RESOLUTION, order=3, mode=mode, cval=0.0)
    
    for idx, img_1 in enumerate(images_sorted[1:]):
        print(f'{idx}th img for stitching...')
#         print("STARTTTTT: ", img_1.affine)
        img_1 = resample_to_output(img_1, TARGET_RESOLUTION, order=3, mode=mode, cval=0.0)
#         img_1 = placing_axes(img_1, img_0.affine.copy(), img_0.header.copy(), [2])
#         print( img_1.affine)
        target_affine = img_0.affine.copy()
        target_affine[2,3] = img_1.affine[2,3].copy()
        target_shape = img_0.shape[:2] + img_1.shape[2:]
#         img_1 = placing_axes(img_1, target_affine, img_0.header.copy(), skip_axis=[2])
        img_1 = resample_from_to(img_1, [target_shape, target_affine])
#         print(img_1.affine)
        img_0 = vol_stitching(img_0, img_1)
#         print(img_0.affine, 'FINISHHHHHHHHH')
        
    return img_0
    
def vol_stitching(im_0, im_1):
    im_0_z = im_0.shape[2]
    im_1_z = im_1.shape[2]

    # calculate overlap region:
    im_0_end = im_0.header['qoffset_z']
    im_1_end = im_1.header['qoffset_z']

    spacing = im_0.header['pixdim'][3]

    im_0_width = im_0_z * spacing
    im_1_width = im_1_z * spacing

    im_1_start = im_1_end + im_1_width
    im_0_start = im_0_end + im_0_width

    overlap = abs(im_0_end - im_1_start)

    overlap_v = int(round(overlap / spacing))

    new_im_dim = abs(round((abs(im_1_end - im_0_start)) / spacing))

    new_img = np.empty([im_0.shape[0], im_0.shape[1], int(new_im_dim)])

    im_0_data = im_0.get_fdata()
    im_1_data = im_1.get_fdata()

    new_img[:, :, 0:(im_1_z - overlap_v)] = im_1_data[:, :, 0:(im_1_z - overlap_v)]
    new_img[:, :, im_1_z:] = im_0_data[:, :, overlap_v:]
    
    # overlap region:
    sigmoid_c = sigmoid(np.linspace(-DEFAULT_LINSPACE, DEFAULT_LINSPACE, overlap_v))
    
    for l in range(0, overlap_v):
        new_img[:, :, (im_1_z - overlap_v + l)] = \
        (1 - sigmoid_c[l]) * im_1_data[:,:, (im_1_z - overlap_v) + l] + (sigmoid_c[l]) * im_0_data[:, :, l]
        
    stitched_img = nb.Nifti1Image(new_img, im_1.affine, im_1.header)
#     placing_axes(stitched_img)
    return stitched_img

def read_ras(file_path, file_type=None, is_label=False):
    _, _, img = file_reader(file_path, file_type)
    img_ras = do_nibabel_transform_to_ras(img)
    if is_label:
        file_labels = list(FILE_TO_LABEL_MAP.keys())
        lidx, labelname = fetch_class_labels_from_filemap(file_path, file_labels)
        return img_ras, lidx, labelname
    else:
        return img_ras

def fetch_class_labels_from_filemap(labelmap_path, file_labels):
    label_idx, label = None, None
    for lidx, file_label in enumerate(file_labels):
        if file_label in labelmap_path.replace(" ", "").upper():
            label_idx, label = lidx, file_label
            break
    return label_idx, label


def placing_axes(vol, target_affine, target_header=None, skip_axis=None):
    
    vol2target = npl.inv(target_affine).dot(vol.affine)
    source_data = vol.get_fdata()
    shifts = tuple(vol2target[:3, 3].astype(np.int32))
    print(shifts)
#     print(source_data.shape)
    for ax, shift in enumerate(shifts):
        if skip_axis is not None and ax in skip_axis:
            continue
        print(ax, shift)
        shift = int(shift)
        if shift < 0:
            source_data = flip_axis(source_data, axis=ax)
        print(-np.abs(shift))
        source_data = np.roll(source_data, -np.abs(shift), axis=ax)
                
    if target_header is None:
        target_header = nb.Nifti1Header()
    stitched_labeled_img = nb.Nifti1Image(source_data, target_affine, target_header)

    return stitched_labeled_img


def label_stitching(label_parts, img_reference):
    header = img_reference.header
    target_affine = img_reference.affine
    steps = header['pixdim'][1:4]
    volume = img_reference.get_fdata()
    im_dim_v = volume.shape
    
    processed_segm = None #np.zeros((500,500,1000))

    im_offx = abs(header['qoffset_x'])
    im_offy = abs(header['qoffset_y'])
    im_offz = abs(header['qoffset_z'])
    q_offsets = [im_offx,im_offy,im_offz]
    
    reference_labelmap = None
    for labelmap_img, lidx, lname in label_parts:
        print(lidx, lname)

        if reference_labelmap is None:
            reference_labelmap = labelmap_img
        else:
            mode = 'nearest'
            labelmap_img = resample_from_to(labelmap_img, [reference_labelmap.shape, reference_labelmap.affine], mode=mode)
            print('c shape:', labelmap_img.shape, labelmap_img.affine)
            
        label_header = labelmap_img.header
        labelmap = labelmap_img.get_fdata()
        labelmap_affine = labelmap_img.affine
        steps_l = label_header['pixdim'][1:4]
        lblmp_offx = abs(label_header['qoffset_x'])
        lblmp_offy = abs(label_header['qoffset_y'])
        lblmp_offz = abs(label_header['qoffset_z'])
        q_offsets_l = [lblmp_offx,lblmp_offy,lblmp_offz]
        
        print(f'vol offsets: {q_offsets}')
        
        print(f'label_offsets: {q_offsets_l}')
        
#         l_vol = np.prod(labelmap.shape)
        segm_dim_v = labelmap.shape
        
        ############
        labelmap2vol = npl.inv(target_affine).dot(labelmap_affine)
        seg_start_inv = np.floor(apply_affine(labelmap2vol, [0,0,0])).astype(np.int32)
        seg_end_inv = apply_affine(labelmap2vol, segm_dim_v).astype(np.int32)
        print("seg start inv v: ",seg_start_inv , "segm end inv v:",seg_end_inv)
        ############
#         shifts = tuple(np.floor(labelmap2vol[:3, 3]).astype(np.int32))
        for ax, shift in enumerate(seg_start_inv):
            print(ax, shift)
            shift = int(shift)
            if True:
                if ax in [0,2]:
                    labelmap = flip_axis(labelmap, axis=ax)
                if shift < 0:
#                     labelmap = flip_axis(labelmap, axis=ax) #if ax == 1 else labelmap
                    seg_end_inv[ax] += abs(seg_start_inv[ax])
                    seg_start_inv[ax] = 0
                else:
#                     labelmap = flip_axis(labelmap, axis=ax)
                    seg_end_inv[ax] -= abs(seg_start_inv[ax])
                    seg_start_inv[ax] = 0
            
        print("updated seg start inv v: ",seg_start_inv , "updated segm end inv v:",seg_end_inv)
        labelmap = np.multiply(lidx, labelmap)
        
        if processed_segm is None:
            seg_vol = np.prod(seg_end_inv)
            vol_vol = np.prod(volume.shape)
            print("inside seg start inv v: ",seg_start_inv , "inside segm end inv v:",seg_end_inv)
            print(f'seg vol: {seg_vol}, vol vol: {vol_vol}')
            processed_segm = np.zeros(seg_end_inv) if seg_vol>vol_vol else np.zeros(volume.shape)
            print("proceess shape:", processed_segm.shape)
            
        processed_segm[seg_start_inv[0]:seg_end_inv[0], seg_start_inv[1]:seg_end_inv[1], seg_start_inv[2]:seg_end_inv[2]] += labelmap
        print("###############################################################################################")

#     print(volume.shape, processed_segm.shape)
#     processed_segm = np.flip(processed_segm)
    labelmap = np.round(processed_segm)
    empty_header = nb.Nifti1Header()
#     processed_segm_img = nb.Nifti1Image(processed_segm, target_affine, empty_header)
#     processed_segm = resample_from_to(processed_segm_img, [volume.shape, target_affine])
    
    
#     volume_ = np.zeros_like(processed_segm)
#     vx, vy, vz = volume.shape
#     volume_[0:vx, 0:vy, 0:vz] = volume
#     volume = volume_
#     volume = np.flip(np.moveaxis(volume, 2, 1))  # 012 -> 021
#     labelmap = np.flip(np.moveaxis(labelmap, 2, 1))
    volume = normalise_data(volume)

    print("shape label:", labelmap.shape)
    stitched_labeled_img = nb.Nifti1Image(labelmap, target_affine, empty_header)
    volume_img = nb.Nifti1Image(volume, target_affine, empty_header)
    
    return volume_img, stitched_labeled_img
    

In [1]:
!conda install jupyterthemes

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.

PackagesNotFoundError: The following packages are not available from current channels:

  - jupyterthemes

Current channels:

  - https://repo.anaconda.com/pkgs/main/osx-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/osx-64
  - https://repo.anaconda.com/pkgs/r/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.




In [2]:
!pip install jupyterthemes

     |████████████████████████████████| 7.0 MB 1.3 MB/s eta 0:00:01
     |████████████████████████████████| 46 kB 2.0 MB/s eta 0:00:01
     |████████████████████████████████| 49 kB 963 kB/s eta 0:00:01


Didn't recognize theme name: dark
Available Themes: 
   chesterish
   grade3
   gruvboxd
   gruvboxl
   monokai
   oceans16
   onedork
   solarizedd
   solarizedl


In [15]:
!jt -t onedork